In [397]:
%pip install -e git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction

Obtaining MI_prediction from git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction
  Updating ./src/mi-prediction clone
  Running command git fetch -q --tags
  Running command git reset --hard -q e437b5386eaec06d85d93b4b66bade33f0d206b4
  Preparing metadata (setup.py) ... done
  Attempting uninstall: MI_prediction
    Found existing installation: MI-prediction 0.1
    Uninstalling MI-prediction-0.1:
      Successfully uninstalled MI-prediction-0.1
  Running setup.py develop for MI_prediction
Note: you may need to restart the kernel to use updated packages.


In [31]:
from MI_prediction.Utils.Datasets import DataLoader, DataLoader_Rest, get_epochs, get_runs
from MI_prediction.Datasets import Cho2017_Rest, BNCI2014001_Rest
from MI_prediction.Utils.Preprocess import filterbank_preprocessor, filterbank,FBCSP
from braindecode.preprocessing.preprocess import exponential_moving_standardize, preprocess, Preprocessor, scale

import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score

from sklearn.preprocessing import StandardScaler
from mne.preprocessing import compute_current_source_density

import time

# Lawhern2018 sklearn

## BCI2a

### Load data

In [28]:
dl = DataLoader(dataset_name="BNCI2014001")
subjects = np.arange(1,10)

### Preprocessing

In [49]:
fb = filterbank_preprocessor([(4.,12.),(12.,38.)])

preprocessors = [
        Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
        Preprocessor(scale, factor=1e6, apply_on_array=True),  # Convert from V to uV
        ]

Ch_prep = [
        Preprocessor(compute_current_source_density,copy=False,apply_on_array=False)
        ]

windows = [
        (0,0)
        ]


band to filter: (4.0, 12.0) Hz
band to filter: (12.0, 38.0) Hz


In [50]:

acc = {}

for s in subjects:
    print ("Subject: {}".format(s))
    tic = time.time()

    dl.load_data(subject_ids=[s])
    ds_f = filterbank(dl, preprocess=preprocessors, filters=fb, channels_prep=Ch_prep)
    trials = [tr.get_trials(start_offset=[0], end_offset=[0]) for tr in ds_f]

    for nw in range(len(windows)):
        epochs = [get_epochs(epoch['win_'+str(nw)].split('session')['session_T']) for epoch in trials]
        
        X = np.concatenate([np.expand_dims(trial[0],axis=3) for trial in epochs], axis=-1,dtype=np.float64)
        y = epochs[0][1]

        idx_l = (y==0)+(y==1)
        X = X[idx_l,:,:,:]
        y = y[idx_l]

        skf = StratifiedKFold(n_splits=4)

        lda = LinearDiscriminantAnalysis()
        fbcsp = FBCSP(n_components=8,norm_trace=False)
        scaler = StandardScaler()
        clf = Pipeline([('CSP', fbcsp), ('LDA', lda)])

        scores = cross_val_score(clf, X, y, cv=skf, n_jobs=-1,verbose=1)

        acc[str(s)] = scores.mean()
        
        print("Accuracy: {}  elapsed: {}".format(scores.mean(), time.time()-tic))


Subject: 1
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
Fitted sphere radius:         85.2 mm
Origin head coordinates:      -0.7 9.9 58.4 mm
Origin device coordinates:    -0.7 9.9 58.4 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Fitted sphere radius:         85.2 mm
Origin head coordinates:      -0.7 9.9 58.4 mm
Origin device coordinates:    -0.7 9.9 58.4 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 413 samples (1.652 sec)

Fitted sphere radius:         85.2 mm
Origin head coordinates:      -0.7 9.9 58.4 mm
Origin device coordinates:    -0.7 9.9 58.4 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causa

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Computing rank from data with rank=None
Computing rank from data with rank=None
Computing rank from data with rank=None
Computing rank from data with rank=None
    Using tolerance 1.2e+04 (2.2e-16 eps * 22 dim * 2.4e+18  max singular value)
    Using tolerance 1.2e+04 (2.2e-16 eps * 22 dim * 2.4e+18  max singular value)
    Using tolerance 1.2e+04 (2.2e-16 eps * 22 dim * 2.5e+18  max singular value)
    Using tolerance 1.2e+04 (2.2e-16 eps * 22 dim * 2.5e+18  max singular value)
    Estimated rank (mag): 22
    Estimated rank (mag): 22
    Estimated rank (mag): 22
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
    MAG: rank 22 computed from 22 data channels with 0 projectors
    MAG: rank 22 computed from 22 data channels with 0 projectors
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Reducing data rank from 22 -> 22
Reducing data rank from 22 -> 22
Reducing data rank from 22 -> 22
Esti

...

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 413 samples (1.652 sec)

Fitted sphere radius:         85.2 mm
Origin head coordinates:      -0.7 9.9 58.4 mm
Origin device coordinates:    -0.7 9.9 58.4 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband att

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Computing rank from data with rank=None
    Using tolerance 2.1e+04 (2.2e-16 eps * 22 dim * 4.3e+18  max singular value)
    Estimated rank (mag): 22
    Using tolerance 2.1e+04 (2.2e-16 eps * 22 dim * 4.3e+18  max singular value)
    MAG: rank 22 computed from 22 data channels with 0 projectors
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
    Using tolerance 2.1e+04 (2.2e-16 eps * 22 dim * 4.3e+18  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Done.
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
Computing rank from data with rank=None
Computing rank from data with rank=None
    Using tolerance 2.1e+04 (2.2e-16 eps * 22 dim * 4.3e+18  max singular value)


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.6s finished


In [51]:
acc (8,12) (12,38)

{'1': 0.923611111111111,
 '2': 0.9166666666666666,
 '3': 0.923611111111111,
 '4': 0.736111111111111,
 '5': 0.6388888888888888,
 '6': 0.7291666666666666,
 '7': 0.9722222222222223,
 '8': 0.7847222222222221,
 '9': 0.763888888888889}

In [42]:
acc # Con laplaciano

{'1': 0.9027777777777778,
 '2': 0.9097222222222222,
 '3': 0.9305555555555556,
 '4': 0.6805555555555556,
 '5': 0.5416666666666666,
 '6': 0.6388888888888888,
 '7': 0.9513888888888888,
 '8': 0.8680555555555556,
 '9': 0.7083333333333333}

In [39]:
acc # sin laplaciano

{'1': 0.9097222222222222,
 '2': 0.8958333333333334,
 '3': 0.9444444444444444,
 '4': 0.7013888888888888,
 '5': 0.5277777777777778,
 '6': 0.6319444444444444,
 '7': 0.9444444444444444,
 '8': 0.8680555555555556,
 '9': 0.7569444444444444}